In [9]:
import os
import openpyxl
import sys

Find the root directory of this workspace

In [10]:
ROOT_DIR = os.path.abspath('../..')

Add to the system path so we can easily import from files elsewhere in this repo.

In [11]:
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

Import processing functions

In [12]:
from lib.ons import (
  read_headers_from_excel_sheet,
  read_data_from_excel_sheet
)

from pipelines.ons.functions import infer_date

Set the input file

In [13]:
X10_RAW_DATA='../../data/raw/x10.xlsx'

Peek into the X10 spreadsheet to see which sheets are published

In [14]:
openpyxl.open(X10_RAW_DATA).sheetnames

['Notes', 'People']

Read the headers and dagta from the sheet

In [15]:
headers = read_headers_from_excel_sheet(X10_RAW_DATA, 'People')

In [47]:
import pandas as pd
data = read_data_from_excel_sheet(X10_RAW_DATA, 'People')
date = data.date.str.slice(4,12).apply(pd.to_datetime, format="%b %Y") - pd.DateOffset(months=1)
data.date = date

Merge them together

In [48]:
x10_data = data.merge(headers.loc[:, ['age', 'measure']], left_index=True, right_index=True).reset_index().drop(columns=['column']).set_index(['date', 'sheet', 'age', 'measure'])

Save the files

In [49]:
headers.to_csv('../../data/metadata/codes/x10-codes.csv')
x10_data.to_csv('../../data/processed/x10.csv')